# 参考项目

https://aistudio.baidu.com/aistudio/projectdetail/4609530?channelType=0&channel=0


# 一.安装PaddleSeg

PaddleSeg是基于飞桨PaddlePaddle的端到端图像分割套件，内置40+模型算法及140+预训练模型，支持配置化驱动和API调用开发方式，打通数据标注、模型开发、训练、压缩、部署的全流程，提供语义分割、交互式分割、Matting、全景分割四大分割能力，助力算法在医疗、工业、遥感、娱乐等场景落地应用。

![](https://ai-studio-static-online.cdn.bcebos.com/05073260b0c3489394026c375b88017e896024aafefb4c6bafb479e8d9236608)



In [ ]:
# 切换到工作目录
%cd /home/aistudio/work

# 解压下载好的PaddleSeg
!unzip PaddleSeg.zip

# 安装PaddleSeg依赖
%cd PaddleSeg
!pip install -r requirements.txt


In [ ]:
# 删除多余文件
!rm -rf /home/aistudio/work/PaddleSeg/data/mine/Image
!rm -rf /home/aistudio/work/PaddleSeg/data/mine/Image_Masks2
!rm -rf /home/aistudio/work/PaddleSeg/data/mine/Layer_Masks
!rm -rf /home/aistudio/work/PaddleSeg/data/mine/test

# 二.数据集处理

PaddleSeg使用单通道的标注图片(灰度标注图)，每一种像素值代表一种类别，像素标注类别需要从0开始递增，例如0，1，2，3表示有4种类别。

建议标注图像使用PNG无损压缩格式的图片，支持的标注类别最多为256类。


通常标注图片是单通道的灰度图，显示是全黑效果，无法直接观察标注是否正确。 在原来的灰度标注图中注入调色板，就可以得到伪彩色的标注图。 灰度标注图和伪彩色标注图的对比如下。

![](https://ai-studio-static-online.cdn.bcebos.com/8057bf7f10864a1d9b8068d9b7a923dafd8006328579471d9e97ae8c9082e657)



In [ ]:
# 解压数据集
%cd /home/aistudio/data/data170514

!unzip -o GOALS2022-Train.zip
!unzip -o GOALS2022-Validation.zip


# 移动到PaddleSeg/data/mine 目录下 - 训练集
! mv Train/Image /home/aistudio/work/PaddleSeg/data/mine/
! mv Train/Layer_Masks /home/aistudio/work/PaddleSeg/data/mine/

# 移动到PaddleSeg/data/mine 目录下 - 测试集
! mv GOALS2022-Validation/Image /home/aistudio/work/PaddleSeg/data/mine/test


生成train.txt和valid.txt 和 灰度标注图

In [ ]:
import cv2
class Pre(object):
    def __init__(self):
        self.dataPath = '/home/aistudio/work/PaddleSeg/data/mine/'
    def run(self):
        # 生成train.txt和valid.txt
        trainFile = open(self.dataPath + 'train.txt', 'w')
        vaildFile = open(self.dataPath + 'valid.txt', 'w')
        for i in range(1,101):
            name = str(i).zfill(4)
            if i <= 90:
                trainFile.write('Image/'+name+'.png Image_Label/'+name+'.png')
                trainFile.write('\n')
            else:
                vaildFile.write('Image/'+name+'.png Image_Label/'+name+'.png')
                vaildFile.write('\n')
            
            # 将标签图片转为标准标准图片
            image = cv2.imread(self.dataPath+'Layer_Masks/'+name+'.png')

            # 像素值为0的是RNFL(类别 0)
            # 像素值为80的是GCIPL(类别 1)
            # 像素值为160的是脉络膜(类别 2)
            # 像素值为255的是其他（类别3）。
            image[image == 80] = 1
            image[image == 160] = 2
            image[image == 255] = 3
            image = image[:,:,1]
            cv2.imwrite(self.dataPath+'Image_Label/'+name+'.png', image)

pre = Pre()
pre.run()

In [ ]:
# 灰度标注图转换为伪彩色标注图 （非必须）
%cd /home/aistudio/work/PaddleSeg

!python tools/gray2pseudo_color.py  data/mine/Image_Label data/mine/Image_Masks2

# 三.训练

本项目选用PP-LiteSeg模型训练学习
![](https://ai-studio-static-online.cdn.bcebos.com/91607bc98acb42b88a50c659b63cdb6b4ffa6dda82ec45548fa65949eba4f6b5)

配置文件在configs/qick_start/mine.yml中。其他模型的配置方案可以参考 configs目录下其他文件

```
batch_size: 10  #设定batch_size的值即为迭代一次送入网络的图片数量，一般显卡显存越大，batch_size的值可以越大。如果使用多卡训练，总得batch size等于该batch size乘以卡数。
iters: 2000    #模型训练迭代的轮数

train_dataset:  #训练数据设置
  type: Dataset #指定加载数据集的类
  dataset_root: data/mine #数据集路径
  train_path: data/mine/train.txt  #数据集中用于训练的标识文件
  num_classes: 4  #指定类别个数（背景也算为一类）
  mode: train #表示用于训练
  transforms: #模型训练的数据预处理方式。
    - type: ResizeStepScaling #将原始图像和标注图像随机缩放为0.5~2.0倍
      min_scale_factor: 0.5
      max_scale_factor: 2.0
      scale_step_size: 0.25
    - type: RandomPaddingCrop #从原始图像和标注图像中随机裁剪512x512大小
      crop_size: [512, 512]
    - type: RandomHorizontalFlip  #对原始图像和标注图像随机进行水平反转
    - type: RandomDistort #对原始图像进行亮度、对比度、饱和度随机变动，标注图像不变
      brightness_range: 0.5
      contrast_range: 0.5
      saturation_range: 0.5
    - type: Normalize #对原始图像进行归一化，标注图像保持不变

val_dataset:  #验证数据设置
  type: Dataset #指定加载数据集的类
  dataset_root: data/mine #数据集路径
  val_path: data/mine/valid.txt  #数据集中用于验证的标识文件
  num_classes: 2  #指定类别个数（背景也算为一类）
  mode: val #表示用于验证
  transforms: #模型验证的数据预处理的方式
    - type: Normalize #对原始图像进行归一化，标注图像保持不变

optimizer: #设定优化器的类型
  type: sgd #采用SGD（Stochastic Gradient Descent）随机梯度下降方法为优化器
  momentum: 0.9 #设置SGD的动量
  weight_decay: 4.0e-5 #权值衰减，使用的目的是防止过拟合

lr_scheduler: # 学习率的相关设置
  type: PolynomialDecay # 一种学习率类型。共支持12种策略
  learning_rate: 0.01 # 初始学习率
  power: 0.9
  end_lr: 0

loss: #设定损失函数的类型
  types:
    - type: CrossEntropyLoss  #CE损失
  coef: [1, 1, 1] # PP-LiteSeg有一个主loss和两个辅助loss，coef表示权重，所以 total_loss = coef_1 * loss_1 + .... + coef_n * loss_n

model:  #模型说明
  type: PPLiteSeg  #设定模型类别
  backbone:  # 设定模型的backbone，包括名字和预训练权重
    type: STDC2
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/PP_STDCNet2.tar.gz
```

In [ ]:
%cd /home/aistudio/work/PaddleSeg/

!export CUDA_VISIBLE_DEVICES=0 # 设置1张可用的卡

!python train.py \
       --config configs/quick_start/mine.yml \
       --save_interval 500 \
       --do_eval \
       --use_vdl \
       --save_dir output

2022-11-16 16:57:59 [INFO]	[TRAIN] epoch: 17, iter: 150/2000, loss: 0.2556, lr: 0.009327, batch_cost: 0.5092, reader_cost: 0.37149, ips: 19.6391 samples/sec | ETA 00:15:41
2022-11-16 16:58:04 [INFO]	[TRAIN] epoch: 18, iter: 160/2000, loss: 0.2452, lr: 0.009282, batch_cost: 0.5189, reader_cost: 0.38357, ips: 19.2714 samples/sec | ETA 00:15:54
2022-11-16 16:58:09 [INFO]	[TRAIN] epoch: 19, iter: 170/2000, loss: 0.2227, lr: 0.009236, batch_cost: 0.5105, reader_cost: 0.37784, ips: 19.5888 samples/sec | ETA 00:15:34
2022-11-16 16:58:14 [INFO]	[TRAIN] epoch: 20, iter: 180/2000, loss: 0.2347, lr: 0.009191, batch_cost: 0.4993, reader_cost: 0.36434, ips: 20.0299 samples/sec | ETA 00:15:08
[ WARN:23@94.859] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('data/mine/Image/0088.png'): can't open/read file: check file path/integrity
Exception in thread Thread-22:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.p

上述训练命令解释：

--config指定配置文件。

--save_interval指定每训练特定轮数后，就进行一次模型保存或者评估（如果开启模型评估）。

--do_eval开启模型评估。具体而言，在训练save_interval指定的轮数后，会进行模型评估。

--use_vdl开启写入VisualDL日志信息，用于VisualDL可视化训练过程。

--save_dir指定模型和visualdl日志文件的保存根路径。


![](https://ai-studio-static-online.cdn.bcebos.com/bdb9a74fdf184b2c874d7aa2748b6d3c106501f7f79346cbbe7a20146333dee8)

![](https://ai-studio-static-online.cdn.bcebos.com/6166129352164112a2c66dd3de286409c6c7aa596b614a539eed845b19521ab0)


# 四.评估

In [ ]:
!python val.py \
       --config configs/quick_start/mine.yml \
       --model_path output/best_model/model.pdparams

# 五.预测生成图片

In [ ]:
!python predict.py \
       --config configs/quick_start/mine.yml \
       --model_path output/best_model/model.pdparams \
       --image_path data/mine/test/Image \
       --save_dir output/result \
       --custom_color 0 0 0 80 80 80 160 160 160 255 255 255

--config 指定配置文件

--model_path 训练中最好的模型

--image_path 带测试图片目录

--save_dir 输出保存的图片路径

--custom_color 分割自定义颜色，赛题要求的颜色值


**OCT层分割结果存储在Layer_Segmentations文件夹中，每个样本的分割结果以png图像格式存储。每个样本对应一个层分割结果图像。分割结果图像命名与输入的待分割OCT图像命名前缀一致。分割图像中，像素值为0代表RNFL区域、像素值为80代表GCIPL区域、像素为160代表脉络膜区域、像素值为255代表其他区域**

最终比赛结果在图片在 output/result/pseudo_color_prediction 目录下

# 六.提交文件格式

目前按照官方格式提交会出现异常，格式如下
![](https://ai-studio-static-online.cdn.bcebos.com/8a7ee8b1da4f41d2aecd0bb6982a6dec16cd33d3087540edb5bdc3839eeaac78)



results.zip 文件放置在 work 根目录，仅供参考

![](https://ai-studio-static-online.cdn.bcebos.com/46a4f94e0c24486d96ec743cb738b9638ccee3e26b6b41d69a8cd4539fae4f92)




# 七.总结

通过PaddleSeg可以解决分割任务，本项目采用的配置文件的方式来训练，PaddleSeg还支持采用API方式，具体请[参考](https://aistudio.baidu.com/aistudio/projectdetail/1339458?channelType=0&channel=0)

优化方向可以通过配置不同的模型（例如 UNet,DeepLab）,微调yml配置来提高精度